<a href="https://colab.research.google.com/github/rutia2044/rutiarbiv-gmail.com/blob/master/Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
 
import torchvision.transforms as transforms
from torchvision.utils import make_grid
 
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
 
import torch.nn as nn
import torch.nn.functional as F
import torch
 
import os
import glob
 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
from tensorflow import keras,summary


In [2]:
import os
from tqdm import tqdm
def Load_images():
  images = []
  root = '/content/drive/MyDrive/gan-getting-started/monet_jpg'
  for file in tqdm(os.listdir(root)):
      images.append(plt.imread(root+'/'+file))
  images = np.array(images)
  return images

FID -calculating the frechet inception distance

In [3]:

import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data

#inputs:
#act1,act2 =The features extractions of 2 groups of images
#the features extractions done by trained model:InceptionV3/VGG..

#output:
#FID- calculated frechet inception distance

def calculate_fid( act1, act2):

	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [6]:
#rand 100 vectors of 30 indexs
def Find_indexes_whith_min_Fid_from_random_vectors(act1, act2):
  min=300
  #max=0
  AllImages = preprocess_input(images)
  for number in range(100):   
      rndArr=np.random.choice(300, 30)
      selectImg=images[rndArr,:,:,:]
      selectImg1 = preprocess_input(selectImg)
      fid = calculate_fid( AllImages, selectImg1)
      if fid < min:
        print('FID (temp min): %.3f' % fid)
        print(number)
        min=fid
        minIndx= rndArr
  print('MIN Final---FID : %.3f' % min)
  print(minIndx)
  return min, minIndx


CLUSTERING

In [7]:
from sklearn.cluster import KMeans
import collections

def Find_indexes_foreach_cluster(act1, act2):
  kmeans = KMeans(n_clusters=30).fit(tsneAllImages)
  collections.Counter(kmeans.labels_)
  select=[]
  for i in range (30):
    select.append(np.nonzero(kmeans.labels_ == i)[0][0])
  return select